### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [18]:
qwerty = pd.read_csv('ml-latest-small/ratings.csv')
qwerty

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [19]:
def classes(stats):
    if stats['rating'] <= 2:
        return 'low'
    elif stats['rating'] <= 4:
        return 'middle'
    elif stats['rating'] <= 5:
        return 'high'

qwerty['class'] = qwerty.apply(classes, axis = 1)
print(qwerty.head())

   userId  movieId  rating   timestamp   class
0       1       31     2.5  1260759144  middle
1       1     1029     3.0  1260759179  middle
2       1     1061     3.0  1260759182  middle
3       1     1129     2.0  1260759185     low
4       1     1172     4.0  1260759205  middle


### Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [26]:
def for_region(stat, region, city):
    return city if city in stat['keyword'] else 'undefined'
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
stat = pd.read_csv('keywords.csv')
for region, cities in geo_data.items():
    for city in cities:
        stat['region'] = stat.apply(for_region, region = region, city = city, axis = 1)

print(stat[stat.region != 'keyword'].head())

         keyword     shows     region
0             вк  64292779  undefined
1  одноклассники  63810309  undefined
2          порно  41747114  undefined
3           ютуб  39995567  undefined
4      вконтакте  21014195  undefined


### Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга



In [27]:
def production_year(stats):
    years = [x for x in range(1950, 2011)]
    new_year = 1900
    for year in years:
        if (str(year) in stats['title']) == True:
            new_year = year
    return new_year

movies = pd.read_csv('ml-latest-small/movies.csv').filter(items = ['movieId', 'title'])
ratings = pd.read_csv('ml-latest-small/ratings.csv').filter(items = ['movieId', 'rating'])
joined = ratings.merge(movies, on='movieId', how='left')
joined['year'] = joined.apply(production_year, axis = 1)

print(joined.groupby('year').agg({'rating': 'mean'}).sort_values(['rating', 'year'], ascending=[False, True]).head(10))


        rating
year          
1957  4.014241
1972  4.011136
1952  4.000000
1974  3.999058
1954  3.994220
1951  3.983539
1962  3.952446
1950  3.915254
1977  3.905786
1964  3.841492
